## proc directory
/proc: directory in charge of linux performance optimization

proc fidrectory has 3 parts
1. PID directory
2. information files /meminfo: prevents information from kernel
3. /sys: tunables

## sys
important interface for sys
- fs: filesystem
- kernel: for kernel
- net: networking
- vm: memory

swappiness: the williness of the kernel to swap
out memory pages, if memory pressure arises

### Systemctl service
```systemctl status systemd-sysct```
its configuration is in /etc/sysctl.conf<br>
changes to system is applied on boot<br>
you can change swappiness

Useful sysctl command

In [ ]:
%%bash
sysctl --help
sysctl -p /etc/sysctl.conf

In [ ]:
%%bash
# list sysctl prameters
sysctl -a

# Managing Kernel Module Parameters
- Use **modinfo** to find which parameters are available
- Use **modprobe module key=value** to specify a parameter in runtime
- Use /etc/modprobe.d/modulename.conf to specify a permanent parameter

## Limiting Resource Usage

### ulimit
ulimit is the old way of configuring resource usage<br>
Applying POSIX Resource Limits
- Set runtime limits with **ulimit**
- - It dapplies to the shell in which the command is used
- Apply persistent ulimit settings to /etc/security/limits.conf
- Soft limits can be modified by the user, hard limits implements an absolute ceiling
- - Users can set soft limits, but only lower than the enforced hard limit

In [ ]:
%%bash
cat /etc/security/limits.conf

#### Managing Persistent limits
- **pam_limits** is applied from PAM sessions to configure persistent limits
- It works with /etc/security/limits.conf and /etc/security/limits.d/*.conf
- See *man 5 limits.conf** for documentation
- - Notice that not all llimites currently work on RHEL. The **rss**limit for instance doesn not work
- Example: set the maximum amount of logins for studentst to 2, creating a file with the contents /etc/security/limits/students.d and the following contents:
- - **@students hard maxlogins 2**

#### Setting Limits to Services
- In systemd units, add the **LIMIT\** entries to the **[Service]** block of a unit file to limit what services can do
- If, for inatance you want to allow your blah.service a maximum of 60 seconds of CPU time before it is killed, add the following to **/etc/systemd/system/blah.service.d/10-cpulimits.conf**

[Service]
LimitCPU=60

- Use **systemctl daemon-reload** and **systemctl restart blah** to make the changes effective
- See **man5 systemd.exec** and **man 2 setrlimit** for a full list of Limit\* settings

PAM (Pluggable Authentication Modules): it shapes what is happening when user log into the system

<img src='screenshots/PAM.png'>

In [ ]:
%%bash
man systemd.exec | grep limits

In [ ]:
! mkdir /etc/systemd/system/sleep.service.d

In [ ]:
%%bash
# to reload
systemctl daemon-reload
systemctl restart
status sleep

## Control Groups
Understand Control Groups
- Control Groups place resources in controllers that represent the type of resource
- Some common default controllers are **cpu**, **memory**, and **blkio**
- These controllers are subdivided in to a tree structure where different weights or limits are applied to each branch
- - Each of these branches is a cgroup
- - One or more processes are assigned to a cgroup
- Cgroups can be applied from the command line, or from systemd
- - Manual creation happened throught the **cgconfig** service and the **cgred** process
- - In all cases, cgroup settings are written to /sys/fs/cgroups

In [ ]:
%%bash
ls /sys/fs/cgroup

## Resource Allocation 
- Machine
- System
- User

<img src='screenshots/CGroup.png'>

### Integrating Cgroups in Systemd
Systemd divides **cpu**, **cpuacct**, **blkio** into slices
- **system** for system processes and daemons
- **machine** for virtual machines
- **user** for user sessions
On a systemd-system, system-enabled cgroups can be omitted and you can still use **cgconfig** and **cgred**. See **systemd-system.conf** for instructions on how to do that and make sure to rebuild initramfs to make this effective

#### Using Custom Slices
- Administrators can create their own slices, naming them *.slice, or slices within a slice, using the **parent-child.slice** naming
- Child slices will inherit the settings of the parent slices
- Make sure to turn on CPU, memory, or I/O accounting to see how they are used within a slice

#### Enabling Accounting
Enabling accounting in the [Service] section of the unit file
- **CPUAccounting=true**
- **MemoryAccounting=true**
- BlockIOAccounting=true**
Or better: enable it in /etc/systemd/system.conf
Use drop-in files to take care of this
- e.g. the SSH service would use a drop-in /etc/systemd/system/sshd.service.d/*conf<br>
**man 5 systemd.resource-control** for all parameters
- CPUShares=512
- MemoryLimit=512M
- BlockIO*=

In [ ]:
%%bash
# listing folder for cpu
ls /sys/devices/system/cpu

```top``` then 1 to check cpu usage

### Managing slice
Putting Commands into a Slice<br>
To put a command into a slice, you can use the **systemd-run** command with the **slice=** option
- **systemd-run --slice=example.slice sleep 10d**
- show with **systemd-cgls /example.slice/<servicename>**
- If the **--slice** option is not used, commands started with **systemd-run** will be put into the **system.slice**  
    
Using Custom Slices
- Put a service into a custom slice using **Slice=my.slice**; if the slice doesn't exist it iwll be created when the service is started
- You can also pre-create custom slices by creating a *.slice file in /etc/systemd/system. Put the tunables in the **[Slice]** section
- To make a lice a child of another slice, git it a name **<parent>-<clild>.slice**; it will inherit all settings of the parent slice
- Note that the slice will only be created once the first process is started within


In [ ]:
! systemd-cgtop

In [ ]:
! systemd-cgls

# Benchmarking
- Benchmarking is comparing performance characteristics to industry standards
- This means that data needs to be gathered and compared
- In IT there are often is no such thing as a standard benchmark, so you'll need to gather and compare a lot 
- Benchmarking is NOT profiling, which is about gathering information about performance hot spots

## Subsystem involved
While benchmarking, different subsystems should be involved
- Processor
- Memory
- Scheduler
- I/O
- Network

## Benchmarking Utilities
- **vmstat**
- - Notive that the first line of vmstat output is giving the average since boot!
- **iostat**
- **mpstat**
- **sar**: can be used for gathering performance data
- **awk**: is useful for data analysis
- **gnuplot**: can be useful for plotting data
- **pcp**: is useful as an extensive testing framework

### Getting Performance data with sar
**sar**
- For filtering purposes, use **LANG=C sar -b**
- Create an alias to do this automatically
- Consider /etc/sysconfig/sysstat for additional settings
- Data collection interval is set through cron

## Gnuplot
you can plot performance data with Gnuplot
